## Installing Requried Libraries

In [1]:
pip install -q -U langchain langchain_openai langchain_google_genai

In [2]:

from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langsmith import Client
from langchain.smith import RunEvalConfig, run_on_dataset

In [3]:
from google.colab import userdata
import os
# LLM API KEY
os.environ["GOOGLE_API_KEY"]=userdata.get('GOOGLE_API_KEY')
os.environ["OPENAI_API_KEY"]=userdata.get('OPENAI_API_KEY')

# configuring langsmith
os.environ["LANGCHAIN_API_KEY"]=userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "LANGSMITH-GUIDE"

In [4]:
# Load the LangSmith Client and Test Run

client = Client()

llm =ChatGoogleGenerativeAI(model="gemini-pro")
output=llm.predict("HI, How are you ? ")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [5]:
# Tracing with langchain
llm.predict("write a poem...")


"**Ode to the Unseen**\n\nIn shadows cast and whispers low,\nA world of secrets, we may not know.\nBeyond our sight, a realm profound,\nWhere mysteries and wonders abound.\n\nThe unseen forces shape our fate,\nGuiding our paths, both small and great.\nSpirits dance in ethereal flight,\nGuarding dreams and guiding our night.\n\nWhispers echo through the silent air,\nMessages from realms beyond compare.\nIntuition's voice, a gentle guide,\nRevealing truths we cannot hide.\n\nIn dreams, we glimpse the unseen land,\nWhere memories and futures intertwine, hand in hand.\nSymbols and visions, cryptic and deep,\nUnveiling secrets we seek to keep.\n\nThough unseen, its presence we can feel,\nIn whispers of the wind, the ocean's swell.\nIn moments of awe and inspiration's fire,\nThe unseen world touches our desire.\n\nSo let us embrace the mystery's embrace,\nAnd seek the wonders hidden in this space.\nFor in the unseen, we find our way,\nTo a realm where dreams and truths hold sway."

In [6]:
# 1. Create a Dataset From a List of Examples (Key-Value Pairs)

example_inputs = [
    ("What is the largest mammal?", "The blue whale"),
    ("What do mammals and birds have in common?", "They are both warm-blooded"),
    ("What are reptiles known for?", "Having scales"),
    (
        "What's the main characteristic of amphibians?",
        "They live both in water and on land",
    ),
]

dataset_name = "Elementary Animal Questions"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Questions and answers about animal phylogenetics.",
)

for input_prompt, output_answer in example_inputs:
    client.create_example(
        inputs={"question": input_prompt},
        outputs={"answer": output_answer},
        dataset_id=dataset.id,
    )

In [10]:
# 2. Create a Dataset From Existing Runs

dataset_name = "Existing Dataset Example"

# Filter runs to add to the dataset
runs = client.list_runs(
    project_name="TEST LLM",
    execution_order=1,
    error=False,
)

dataset = client.create_dataset(dataset_name, description="An example dataset")

for run in runs:
    client.create_example(
        inputs=run.inputs,
        outputs=run.outputs,
        dataset_id=dataset.id,
    )

In [12]:
# 3. Create a Dataset From a Dataframe

# Create a Dataframe
import pandas as pd

example_inputs = [
    ("What is the largest mammal?", "The blue whale"),
    ("What do mammals and birds have in common?", "They are both warm-blooded"),
    ("What are reptiles known for?", "Having scales"),
    (
        "What's the main characteristic of amphibians?",
        "They live both in water and on land",
    ),
]

df_dataset = pd.DataFrame(example_inputs, columns=["Question", "Answer"])
df_dataset.head()

,Question,Answer
0,What is the largest mammal?,The blue whale
1,What do mammals and birds have in common?,They are both warm-blooded
2,What are reptiles known for?,Having scales
3,What's the main characteristic of amphibians?,They live both in water and on land


In [14]:
input_keys = ["Question"]
output_keys = ["Answer"]

# Create Dataset

dataset = client.upload_dataframe(
    df=df_dataset,
    input_keys=input_keys,
    output_keys=output_keys,
    name="Dataframe",
    description="Dataset created from a dataframe",
    data_type="kv", )

In [17]:
# 4. Create a Dataset From a CSV File

# Save the Dataframe as a CSV File

csv_path = "/content/Churn_Modelling.csv"
df_dataset.to_csv(csv_path, index=False)

# Create Dataset

dataset = client.upload_csv(
    csv_file=csv_path,
    input_keys=input_keys,
    output_keys=output_keys,
    name="CSV Dataset Example",
    description="Dataset created from a CSV file",
    data_type="kv",
)

In [19]:
# 1. Create a Dataset (Only Inputs, No Output)

example_inputs = [
    "a rap battle between Atticus Finch and Cicero",
    "a rap battle between Barbie and Oppenheimer",
    "a Pythonic rap battle between two swallows: one European and one African",
    "a rap battle between Aubrey Plaza and Stephen Colbert",
]

dataset_name = "Battle Dataset"

# Storing inputs in a dataset lets us
# run chains and LLMs over a shared set of examples.
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Rap battle prompts.",
)

for input_prompt in example_inputs:
    # Each example must be unique and have inputs defined.
    # Outputs are optional
    client.create_example(
        inputs={"question": input_prompt},
        outputs=None,
        dataset_id=dataset.id,
    )

In [20]:
# 2. Evaluate Datasets with LLM

eval_config = RunEvalConfig(
    evaluators=[
        # You can specify an evaluator by name/enum.
        # In this case, the default criterion is "helpfulness"
        "criteria",
        # Or you can configure the evaluator
        RunEvalConfig.Criteria("harmfulness"),
        RunEvalConfig.Criteria("misogyny"),
        RunEvalConfig.Criteria(
            {
                "cliche": "Are the lyrics cliche? "
                "Respond Y if they are, N if they're entirely unique."
            }
        ),
    ]
)

run_on_dataset(
    client=client,
    dataset_name=dataset_name,
    llm_or_chain_factory=llm,
    evaluation=eval_config,
)

View the evaluation results for project 'helpful-grass-14' at:
https://smith.langchain.com/o/58fe5e41-0da9-5dcc-b9ac-d4448bfd58f9/datasets/60a59af6-1206-4b53-a4bf-b39f9d97e289/compare?selectedSessions=2f5a12ac-44f4-4f8b-87fb-122d76c82c7a

View all tests for Dataset Battle Dataset at:
https://smith.langchain.com/o/58fe5e41-0da9-5dcc-b9ac-d4448bfd58f9/datasets/60a59af6-1206-4b53-a4bf-b39f9d97e289
[>                                                 ] 0/4

Error Type: StopCandidateException, Message: finish_reason: SAFETY
index: 0
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: MEDIUM
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}



[------------------------------------------------->] 4/4

{'project_name': 'helpful-grass-14',
 'results': {'39ba7895-774e-4b20-a601-39397a033ac0': {'input': {'question': 'a rap battle between Aubrey Plaza and Stephen Colbert'},
   'feedback': [],
   'execution_time': 11.270944,
   'run_id': '49f74399-6703-4dd3-9e25-819747b11880',
   'Error': google.generativeai.types.generation_types.StopCandidateException(finish_reason: SAFETY
                                                                     index: 0
                                                                     safety_ratings {
                                                                       category: HARM_CATEGORY_SEXUALLY_EXPLICIT
                                                                       probability: NEGLIGIBLE
                                                                     }
                                                                     safety_ratings {
                                                                       category: HARM_CATEGORY_H

In [21]:
# 1. Evaluate Datasets That Contain Labels

evaluation_config = RunEvalConfig(
    evaluators=[
        "qa",  # correctness: right or wrong
        "context_qa",  # refer to example outputs
        "cot_qa",  # context_qa + reasoning
    ]
)

run_on_dataset(
    client=client,
    dataset_name="Elementary Animal Questions",
    llm_or_chain_factory=llm,
    evaluation=evaluation_config,
)

View the evaluation results for project 'weary-drink-63' at:
https://smith.langchain.com/o/58fe5e41-0da9-5dcc-b9ac-d4448bfd58f9/datasets/8b476106-55ff-4cf9-8ee0-696afee5a3a7/compare?selectedSessions=8b9b3e9a-fb40-41d2-a03d-ea6272eb759f

View all tests for Dataset Elementary Animal Questions at:
https://smith.langchain.com/o/58fe5e41-0da9-5dcc-b9ac-d4448bfd58f9/datasets/8b476106-55ff-4cf9-8ee0-696afee5a3a7
[------------------------------------------------->] 4/4

{'project_name': 'weary-drink-63',
 'results': {'5f4ee49e-ac38-4dc3-98e5-c48a4763c9fb': {'input': {'question': "What's the main characteristic of amphibians?"},
   'feedback': [EvaluationResult(key='correctness', score=None, value=None, comment="Error evaluating run 17834c18-e677-40b3-a48c-0e57df1ec2f5: Error code: 401 - {'error': {'message': 'Incorrect API key provided: asjk. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}", correction=None, evaluator_info={}, feedback_config=None, source_run_id=None, target_run_id=None),
    EvaluationResult(key='Contextual Accuracy', score=None, value=None, comment="Error evaluating run 17834c18-e677-40b3-a48c-0e57df1ec2f5: Error code: 401 - {'error': {'message': 'Incorrect API key provided: asjk. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}

In [22]:
# 2. Evaluate Datasets With Customized Criterias

evaluation_config = RunEvalConfig(
    evaluators=[
        # You can define an arbitrary criterion as a key: value pair in the criteria dict
        RunEvalConfig.LabeledCriteria(
            {
                "helpfulness": (
                    "Is this submission helpful to the user,"
                    " taking into account the correct reference answer?"
                )
            }
        ),
    ]
)

run_on_dataset(
    client=client,
    dataset_name="Elementary Animal Questions",
    llm_or_chain_factory=llm,
    evaluation=evaluation_config,
)

View the evaluation results for project 'left-representative-33' at:
https://smith.langchain.com/o/58fe5e41-0da9-5dcc-b9ac-d4448bfd58f9/datasets/8b476106-55ff-4cf9-8ee0-696afee5a3a7/compare?selectedSessions=c5ab7bf4-a629-4825-a53c-c3ef36ec1331

View all tests for Dataset Elementary Animal Questions at:
https://smith.langchain.com/o/58fe5e41-0da9-5dcc-b9ac-d4448bfd58f9/datasets/8b476106-55ff-4cf9-8ee0-696afee5a3a7
[------------------------------------------------->] 4/4

{'project_name': 'left-representative-33',
 'results': {'5f4ee49e-ac38-4dc3-98e5-c48a4763c9fb': {'input': {'question': "What's the main characteristic of amphibians?"},
   'feedback': [EvaluationResult(key='helpfulness', score=None, value=None, comment="Error evaluating run dd7f5298-6465-4b38-886a-e954d60fcf6c: Error code: 401 - {'error': {'message': 'Incorrect API key provided: asjk. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}", correction=None, evaluator_info={}, feedback_config=None, source_run_id=None, target_run_id=None)],
   'execution_time': 2.218619,
   'run_id': 'dd7f5298-6465-4b38-886a-e954d60fcf6c',
   'output': AIMessage(content='Amphibians are vertebrates that are adapted to live both in water and on land. They have smooth, moist skin that allows them to absorb oxygen through their skin, and they lay their eggs in water. Amphibians include frogs, toads, salamanders,

In [23]:
# 3. Evaluate Datasets Without Labels

evaluation_config = RunEvalConfig(
    evaluators=[
        # You can define an arbitrary criterion as a key: value pair in the criteria dict
        RunEvalConfig.Criteria(
            {"creativity": "Is this submission creative, imaginative, or novel?"}
        ),
        # We provide some simple default criteria like "conciseness" you can use as well
        RunEvalConfig.Criteria("conciseness"),
    ]
)

run_on_dataset(
    client=client,
    dataset_name="Rap Battle Dataset",
    llm_or_chain_factory=llm,
    evaluation=evaluation_config,
)

View the evaluation results for project 'reflecting-knowledge-72' at:
https://smith.langchain.com/o/58fe5e41-0da9-5dcc-b9ac-d4448bfd58f9/datasets/62bca44a-ec1a-4d8b-8497-484d5daa279a/compare?selectedSessions=8c67cf60-c619-41ad-a96e-866f289fb18f

View all tests for Dataset Rap Battle Dataset at:
https://smith.langchain.com/o/58fe5e41-0da9-5dcc-b9ac-d4448bfd58f9/datasets/62bca44a-ec1a-4d8b-8497-484d5daa279a
[------------------------------------------------->] 4/4

{'project_name': 'reflecting-knowledge-72',
 'results': {'1744321a-02e5-4083-96e5-2cba47918741': {'input': {'question': 'a rap battle between Aubrey Plaza and Stephen Colbert'},
   'feedback': [EvaluationResult(key='creativity', score=None, value=None, comment="Error evaluating run cf65a4d9-099f-4c9c-bfd0-73a838138cf6: Error code: 401 - {'error': {'message': 'Incorrect API key provided: asjk. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}", correction=None, evaluator_info={}, feedback_config=None, source_run_id=None, target_run_id=None),
    EvaluationResult(key='conciseness', score=None, value=None, comment="Error evaluating run cf65a4d9-099f-4c9c-bfd0-73a838138cf6: Error code: 401 - {'error': {'message': 'Incorrect API key provided: asjk. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_a

In [24]:
# 4. Evaluate Datasets Based on Cosine Distance Criteria
# Cosine Distance: Ranged Between 0 to 1. 0 = More Similar

evaluation_config = RunEvalConfig(
    evaluators=[
        # You can define an arbitrary criterion as a key: value pair in the criteria dict
        "embedding_distance",
        # Or to customize the embeddings:
        # Requires 'pip install sentence_transformers'
        # RunEvalConfig.EmbeddingDistance(embeddings=HuggingFaceEmbeddings(), distance_metric="cosine"),
    ]
)

run_on_dataset(
    client=client,
    dataset_name="Elementary Animal Questions",
    llm_or_chain_factory=llm,
    evaluation=evaluation_config,
)

View the evaluation results for project 'fresh-toe-99' at:
https://smith.langchain.com/o/58fe5e41-0da9-5dcc-b9ac-d4448bfd58f9/datasets/8b476106-55ff-4cf9-8ee0-696afee5a3a7/compare?selectedSessions=c12da795-bc3b-4b73-b2ee-bd32d65fbbb6

View all tests for Dataset Elementary Animal Questions at:
https://smith.langchain.com/o/58fe5e41-0da9-5dcc-b9ac-d4448bfd58f9/datasets/8b476106-55ff-4cf9-8ee0-696afee5a3a7
[>                                                 ] 0/4

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


[------------------------------------------------->] 4/4

{'project_name': 'fresh-toe-99',
 'results': {'5f4ee49e-ac38-4dc3-98e5-c48a4763c9fb': {'input': {'question': "What's the main characteristic of amphibians?"},
   'feedback': [EvaluationResult(key='embedding_cosine_distance', score=None, value=None, comment="Error evaluating run 320f4d88-fcc0-46ad-853e-356311b3d543: Error code: 401 - {'error': {'message': 'Incorrect API key provided: asjk. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}", correction=None, evaluator_info={}, feedback_config=None, source_run_id=None, target_run_id=None)],
   'execution_time': 2.897024,
   'run_id': '320f4d88-fcc0-46ad-853e-356311b3d543',
   'output': AIMessage(content='Amphibians are vertebrates that live both in water and on land. They have smooth, moist skin that is covered in mucus, which helps them to absorb oxygen and water. Amphibians also have lungs, which they use to breathe air. They have webb